# RAG 기반 쇼핑몰 챗봇 시스템 데모

이 노트북에서는 RAG 기반 쇼핑몰 고객 서비스 챗봇의 주요 기능들을 테스트하고 시연합니다.

## 주요 구성 요소
1. **Intent Classifier**: 사용자 의도 분류
2. **RAG Processor**: FAQ 및 상품 정보 검색
3. **DB Query Engine**: 주문/사용자 정보 조회
4. **Delivery API Wrapper**: 배송 추적
5. **Response Styler**: 응답 스타일링

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
llm.invoke("Hello, world!")


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BmF8bcPY6yRGdmskDLfycA4NPYw9O', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--87ff9d05-d103-4ddb-a208-196a0362a6e0-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# 필요한 라이브러리 임포트
import sys
from pathlib import Path

# 프로젝트 루트를 Python 경로에 추가
project_root = Path().absolute().parent
sys.path.append(str(project_root))

print(f"프로젝트 루트: {project_root}")

In [ ]:
# 핵심 모듈들 임포트
from core.intent_classifier import IntentClassifier, IntentType
from core.rag_processor import RAGProcessor
from core.db_query_engine import DatabaseQueryEngine
from core.delivery_api_wrapper import DeliveryAPIWrapper
from core.response_styler import ResponseStyler, ResponseTone

print("✅ 모든 모듈 임포트 완료")

## 1. 의도 분류기 테스트

In [ ]:
# 의도 분류기 초기화
intent_classifier = IntentClassifier()

# 테스트 질문들
test_questions = [
    "배송비는 얼마인가요?",
    "무선 이어폰 사양이 어떻게 되나요?",
    "내 주문 ORD20241201001 상태 알려주세요",
    "운송장번호 123456789012 배송 현황 확인해주세요",
    "반품은 어떻게 하나요?",
    "안녕하세요",
    "이상한 질문입니다"
]

print("🔍 의도 분류 테스트 결과:")
print("=" * 50)

for question in test_questions:
    result = intent_classifier.classify(question)
    strategy = intent_classifier.get_processing_strategy(result)
    
    print(f"질문: {question}")
    print(f"의도: {result.intent.value} (신뢰도: {result.confidence:.2f})")
    print(f"엔티티: {result.entities}")
    print(f"처리 전략: {strategy}")
    print("-" * 30)

## 2. RAG 프로세서 테스트

In [ ]:
# RAG 프로세서 초기화
rag_processor = RAGProcessor()
rag_processor.initialize_vector_store()

print("✅ RAG 프로세서 초기화 완료")

In [ ]:
# FAQ 및 상품 정보 검색 테스트
rag_test_queries = [
    "배송비는 얼마인가요?",
    "무선 이어폰 배터리 지속시간은?",
    "반품 절차를 알려주세요",
    "스마트 워치 방수 기능이 있나요?"
]

print("📚 RAG 검색 테스트 결과:")
print("=" * 50)

for query in rag_test_queries:
    result = rag_processor.process_query(query)
    
    print(f"질문: {query}")
    print(f"답변: {result['response'][:200]}...")
    print(f"신뢰도: {result['confidence']:.2f}")
    print(f"소스 수: {len(result['sources'])}")
    
    if result['sources']:
        print("소스 정보:")
        for i, source in enumerate(result['sources'][:2], 1):
            print(f"  {i}. {source['source']} - {source['content_preview'][:50]}...")
    
    print("-" * 30)

## 3. 데이터베이스 쿼리 엔진 테스트

In [ ]:
# DB 쿼리 엔진 초기화
db_engine = DatabaseQueryEngine()

print("🗄️ 데이터베이스 쿼리 테스트:")
print("=" * 50)

# 주문 조회 테스트
print("1. 주문 조회 테스트:")
order = db_engine.get_order_by_id("ORD20241201001")
if order:
    print("✅ 주문 조회 성공")
    print(db_engine.format_order_info(order))
else:
    print("❌ 주문 조회 실패 - 데이터베이스를 초기화해주세요")

print("\n2. 사용자 주문 목록 테스트:")
orders = db_engine.get_recent_orders_by_phone("010-1234-5678")
if orders:
    print("✅ 사용자 주문 목록 조회 성공")
    print(db_engine.format_user_orders(orders))
else:
    print("❌ 사용자 주문 목록 조회 실패")

print("\n3. 상품 검색 테스트:")
products = db_engine.search_products("이어폰")
if products:
    print("✅ 상품 검색 성공")
    for product in products[:3]:
        print(f"- {product['name']} ({product['price']:,}원)")
else:
    print("❌ 상품 검색 실패")

## 4. 배송 API 래퍼 테스트

In [ ]:
# 배송 API 래퍼 초기화
delivery_api = DeliveryAPIWrapper()

print("🚚 배송 API 테스트:")
print("=" * 50)

# 배송 추적 테스트
print("1. 배송 추적 테스트:")
tracking_info = delivery_api.track_package("123456789012")
if tracking_info:
    print("✅ 배송 추적 성공")
    print(delivery_api.format_delivery_info(tracking_info))
else:
    print("❌ 배송 추적 실패")

print("\n2. 배송 예상 시간 테스트:")
estimate = delivery_api.get_delivery_estimate("서울", "부산")
print(f"✅ 배송 예상: {estimate['description']}")

print("\n3. 배송 가능 지역 테스트:")
availability = delivery_api.check_delivery_availability("제주시 연동")
print(f"✅ 배송 가능: {availability['available']}")
if availability.get('note'):
    print(f"참고사항: {availability['note']}")

## 5. 응답 스타일러 테스트

In [ ]:
# 응답 스타일러 초기화
response_styler = ResponseStyler()

print("✨ 응답 스타일러 테스트:")
print("=" * 50)

# 다양한 톤으로 스타일링 테스트
test_responses = [
    ("배송비는 5만원 이상 주문시 무료입니다.", ResponseTone.FRIENDLY),
    ("주문번호 ORD20241201001의 상태는 배송중입니다.", ResponseTone.INFORMATIVE),
    ("죄송합니다. 해당 정보를 찾을 수 없습니다.", ResponseTone.APOLOGETIC),
    ("상품 사양은 다음과 같습니다.", ResponseTone.HELPFUL)
]

for i, (response, tone) in enumerate(test_responses, 1):
    styled = response_styler.style_response(response, tone, include_greeting=True)
    
    print(f"{i}. {tone.value.upper()} 톤:")
    print(f"원본: {response}")
    print(f"스타일링: {styled}")
    print("-" * 30)

## 6. 통합 챗봇 시뮬레이션

In [ ]:
# 통합 챗봇 시뮬레이션 함수
def simulate_chatbot_conversation(user_input):
    """챗봇 대화 시뮬레이션"""
    print(f"👤 사용자: {user_input}")
    
    # 1. 의도 분류
    intent_result = intent_classifier.classify(user_input)
    strategy = intent_classifier.get_processing_strategy(intent_result)
    
    print(f"🧠 의도: {intent_result.intent.value} (신뢰도: {intent_result.confidence:.2f})")
    print(f"⚙️ 처리 전략: {strategy}")
    
    # 2. 전략에 따른 응답 생성
    if strategy == "rag_processor":
        result = rag_processor.process_query(user_input)
        response = result['response']
    elif strategy == "db_query":
        entities = intent_result.entities
        if 'order_id' in entities:
            order = db_engine.get_order_by_id(entities['order_id'])
            response = db_engine.format_order_info(order) if order else "주문을 찾을 수 없습니다."
        else:
            response = "주문 조회를 위해 주문번호를 알려주세요."
    elif strategy == "delivery_api":
        entities = intent_result.entities
        if 'tracking_number' in entities:
            delivery_info = delivery_api.track_package(entities['tracking_number'])
            response = delivery_api.format_delivery_info(delivery_info) if delivery_info else "배송 정보를 찾을 수 없습니다."
        else:
            response = "배송 조회를 위해 운송장번호를 알려주세요."
    else:
        response = "안녕하세요! 무엇을 도와드릴까요?"
    
    # 3. 응답 스타일링
    tone = response_styler.determine_tone(intent_result.intent.value, intent_result.confidence)
    styled_response = response_styler.style_response(response, tone, include_greeting=True)
    
    print(f"🤖 챗봇: {styled_response}")
    print("=" * 80)

# 대화 시뮬레이션
conversation_examples = [
    "안녕하세요!",
    "배송비는 얼마인가요?",
    "무선 이어폰 배터리 시간이 궁금해요",
    "주문번호 ORD20241201001 상태 확인해주세요",
    "운송장번호 123456789012 배송 현황 알려주세요"
]

print("💬 챗봇 대화 시뮬레이션:")
print("=" * 80)

for example in conversation_examples:
    simulate_chatbot_conversation(example)

## 결론

이 노트북에서는 RAG 기반 쇼핑몰 챗봇 시스템의 주요 구성 요소들을 테스트했습니다:

1. ✅ **의도 분류기**: 사용자 질문의 의도를 정확히 분류
2. ✅ **RAG 프로세서**: FAQ와 상품 정보를 효과적으로 검색
3. ✅ **DB 쿼리 엔진**: 주문과 사용자 정보를 정확히 조회
4. ✅ **배송 API**: 배송 추적 정보를 제공
5. ✅ **응답 스타일러**: 일관되고 친근한 응답 생성

모든 구성 요소가 정상적으로 작동하며, 통합된 챗봇 시스템이 다양한 고객 문의에 적절히 응답할 수 있음을 확인했습니다.

### 다음 단계
- Streamlit 앱 실행: `streamlit run app/main.py`
- 추가 테스트 및 성능 최적화
- 실제 운영 환경 배포 준비